## Computer Vision
- Image Classification
  - ImageNet
  - Anomaly Detection
  - Out of Distributions
- Object Detection
  - Fast R-CNN
  - YOLO
- Image Segmentation
  - Fully Convolutional Networks(FCN)
  - UNet
- Imgae Generation
  - Generative Models(GAN ... )
  - Super Resolution

### VGGNET
- 기존 네트워크들이 5X5, 7X7 Conv.layer 이용
- 3X3 Conv.layer를 반복사용하여 5X5, 7X7 layer를 대체
  - 3X3 * 2 -> 5X5 대체
  - 3X3 * 3 -> 7X7 대체
- 이를 통해 적은 W로 깊은 네트워크 구성

### ResNET
- Residual Connection 이용
- ImageNet 대회에서 깊은 네트워크가 우승을 차지
  - Gradient Vanishing 및 최적화 문제 발생
  - 데이터의 복잡도에 따라서 최적의 깊이도 존재
  - 만약 30개의 레이어를 쌓았는데, 20개가 최적이라면? 10개는 y=x 레이어로 만들면 될까? (identity 함수)  
  
#### Identity Function
- F(x) = H(x) - x
- H(x) = F(x) + x, F(x) = 0?
- => Residual Connection은 Gradient Vanishing을 방지하는 방법

## Transfer Learning
- 각 레이어는 Feature를 추출하는 역할을 함
  - conv.layer는 위치에 따라 low-level 또는 high-level feature를 추출
  - 데이터가 다르더라도 이미지를 활용한 task에는 공통된 feature이 존재할 것이라 가정
- Big Dataset -> Load Weights -> Fine-tuning

In [3]:
# import requests 
# from os import listdir, mkdir, rename
# from os.path import isfile, isdir, join
# from zipfile import ZipFile


# def download_url(url, save_path, chunk_size=128):
#     r = requests.get(url, stream=True)
#     with open(save_path, 'wb') as fd:
#         for chunk in r.iter_content(chunk_size=chunk_size):
#             fd.write(chunk)
#     print('Downloading zip file completed.')


# def unzip(zip_path, dataset_path):
#     zf = ZipFile(zip_path)
#     zf.extractall(path=dataset_path)
#     zf.close()
#     print('Unzipping completed.')


# def restructure_dir(data_path, is_train=True):
#     files = [f for f in listdir(data_path) if isfile(join(data_path, f))]

#     if is_train:
#         for file in files:
#             if not isdir(join(data_path, file.split('.')[0])):
#                 mkdir(join(data_path, file.split('.')[0]))
#             rename(
#                 join(data_path, file), join(data_path, file.split('.')[0], file)
#             )
#     else:
#         for file in files:
#             if not isdir(join(data_path, 'dummy')):
#                 mkdir(join(data_path, 'dummy'))
#             rename(
#                 join(data_path, file), join(data_path, 'dummy', file)
#             )
#     print('Resturcturing completed.')


# if __name__ == '__main__':

#     # make dataset directory
#     dataset_path = './dataset'
#     if not isdir(dataset_path):
#         print('Making dataset directory on {}'.format(dataset_path))
#         mkdir(dataset_path)

#     # set hymenoptera dataset
#     hymenoptera_url = 'https://download.pytorch.org/tutorial/hymenoptera_data.zip'
#     hymenoptera_path = './hymenoptera.zip'

#     download_url(hymenoptera_url, hymenoptera_path)
#     unzip(hymenoptera_path, dataset_path)
#     rename(join(dataset_path, 'hymenoptera_data'), join(dataset_path, 'hymenoptera'))
#     rename(join(dataset_path, 'hymenoptera', 'val'), join(dataset_path, 'hymenoptera', 'test'))

Making dataset directory on ./dataset
Unzipping completed.


In [26]:
import os

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, random_split

from torchvision import transforms
from torchvision.datasets import ImageFolder

In [22]:

def load_dataset(
    data_transforms,
    dataset_dir='./dataset',
    dataset_name='catdog',
    is_train=True):
    dataset_dir = os.path.join(dataset_dir, dataset_name)
    is_train_dir = 'train' if is_train else 'test'

    dataset = ImageFolder(
        os.path.join(dataset_dir, is_train_dir),
        data_transforms[is_train_dir]
    )
    return dataset

def divide_dataset(
    train_set,
    test_set,
    data_name='catdog',
    train_ratio=.6,
    valid_ratio=.2,
    test_ratio=.2):
    if data_name == 'catdog':
        train_cnt = int(len(train_set) * train_ratio)
        valid_cnt = int(len(train_set) * valid_ratio)
        test_cnt = len(train_set) - train_cnt - valid_cnt

        train_set, valid_set, test_set = random_split(
            train_set,
            [train_cnt, valid_cnt, test_cnt]
        )
    elif data_name == 'hymenoptera':
        valid_ratio = valid_ratio / (train_ratio + valid_ratio)
        valid_cnt = int(len(train_set) * valid_ratio)
        train_cnt = len(train_set) - valid_cnt

        train_set, valid_set = random_split(
            train_set,
            [train_cnt, valid_cnt]
        )
    else:
        raise NotImplementedError('You need to specify dataset name.')

    return train_set, valid_set, test_set

def get_loaders(config, input_size):
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
        ]),
    }
    
    dataset_name = config.dataset_name
    train_set = load_dataset(
        data_transforms, dataset_name=dataset_name, is_train=True
    )
    test_set = load_dataset(
        data_transforms, dataset_name=dataset_name, is_train=False
    )

    # Shuffle dataset to split into valid/test set.
    train_set, valid_set, test_set = divide_dataset(
        train_set, test_set, config.dataset_name,
        config.train_ratio, config.valid_ratio, config.test_ratio
    )

    train_loader = DataLoader(
        dataset=train_set,
        batch_size=config.batch_size,
        shuffle=True,
    )
    valid_loader = DataLoader(
        dataset=valid_set,
        batch_size=config.batch_size,
        shuffle=False,
    )
    test_loader = DataLoader(
        dataset=test_set,
        batch_size=config.batch_size,
        shuffle=False,
    )

    return train_loader, valid_loader, test_loader


In [32]:
from copy import deepcopy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils as torch_utils

from ignite.engine import Engine
from ignite.engine import Events
from ignite.metrics import RunningAverage
from ignite.contrib.handlers.tqdm_logger import ProgressBar

from module_ignite.utils import get_grad_norm, get_parameter_norm


VERBOSE_EPOCH_WISE=1
VERBOSE_BATCH_WISE=1

class MyEngine(Engine):

    def __init__(self, func, model, crit, optimizer, config):
        self.model = model
        self.crit = crit
        self.optimizer = optimizer
        self.config = config

        super().__init__(func)

        self.best_loss = np.inf
        self.best_model = None

        self.device = next(model.parameters()).device

    @staticmethod
    def train(engine, mini_batch):
        engine.model.train()
        engine.optimizer.zero_grad()
        x, y = mini_batch
        x, y = x.to(engine.device), y.to(engine.device)

        y_hat = engine.model(x)

        loss = engine.crit(y_hat, y)
        loss.backward()

        # Classification인지 확인
        if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
            accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
        else :
            accuracy = 0

        # parameter L2 Norm, Model parameter 학습될수록 높아짐
        p_norm = float(get_parameter_norm(engine.model.parameters()))
        # Gradient L2 Norm, loss surface의 가파름 정도
        g_norm = float(get_grad_norm(engine.model.parameters()))

        engine.optimizer.step()

        return {
            'loss': float(loss),
            'accuracy': float(accuracy),
            '|param|': p_norm,
            '|g_param|': g_norm,
        }

    @staticmethod
    def validate(engine, mini_batch):
        engine.model.eval()

        with torch.no_grad():
            x, y = mini_batch
            x, y = x.to(engine.device), y.to(engine.device)

            y_hat = engine.model(x)

            loss = engine.crit(y_hat, y)

            if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
                accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
            else :
                accuracy = 0

        return {
            'loss': float(loss),
            'accuracy': float(accuracy)
        }

    
    @staticmethod
    def attach(train_engine, validation_engine, verbose=VERBOSE_BATCH_WISE):

        def attach_running_average(engine, metric_name):
            RunningAverage(output_transform=lambda x: x[metric_name]).attach(
                engine,
                metric_name
            )

        training_metric_names = ['loss', 'accuracy', '|param|', '|g_param|']

        for metric_name in training_metric_names:
            attach_running_average(train_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(train_engine, training_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @train_engine.on(Events.EPOCH_COMPLETED)
            def print_train_logs(engine):
                print('Epoch {} - |param|={:.2e} |g-param|={:.2e} loss={:.4e} accuracy={:.4f}'.format(
                    engine.state.epoch,
                    engine.state.metrics['|param|'],
                    engine.state.metrics['|g_param|'],
                    engine.state.metrics['loss'],
                    engine.state.metrics['accuracy'],
                ))

        validation_metric_names = ['loss', 'accuracy']

        for metric_name in validation_metric_names:
            attach_running_average(validation_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(validation_engine, validation_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @validation_engine.on(Events.EPOCH_COMPLETED)
            def print_validation_logs(engine):
                print('Validation - loss={:.4e} accuracy={:.4f} best_loss={:.4e}'.format(
                    engine.state.metrics['loss'],
                    engine.state.metrics['accuracy'],
                    engine.best_loss,
                ))

    @staticmethod
    def check_best(engine):
        loss = float(engine.state.metrics['loss'])
        if loss <= engine.best_loss:
            engine.best_loss = loss
            engine.best_model = deepcopy(engine.model.state_dict())

    @staticmethod
    def save_model(engine, train_engine, config, **kwargs):
        torch.save({
            'model': engine.best_model,
            'config': config,
            **kwargs
        }, config.model_fn)



class Trainer_ignite():

    def __init__(self, config):
        self.config = config

    def train(self,
              model, crit, optimizer,
              train_loader , valid_loader):
        train_engine = MyEngine(
            MyEngine.train,
            model, crit, optimizer, self.config
        )
        validation_engine = MyEngine(
            MyEngine.validate,
            model, crit, optimizer, self.config
        )

        MyEngine.attach(
            train_engine,
            validation_engine,
            verbose=self.config.verbose
        )

        def run_validation(engine, validation_engine, valid_loader):
            validation_engine.run(valid_loader, max_epochs=1)

        train_engine.add_event_handler(
            Events.EPOCH_COMPLETED,
            run_validation,
            validation_engine, valid_loader,
        )
        validation_engine.add_event_handler(
            Events.EPOCH_COMPLETED,
            MyEngine.check_best,
        )
        validation_engine.add_event_handler(
            Events.EPOCH_COMPLETED,
            MyEngine.save_model,
            train_engine, self.config
        )

        train_engine.run(
            train_loader,
            max_epochs=self.config.n_epochs
        )

        return model

In [34]:
class Config():
    def __init__(self):
        self.model_fn = 'model3.pth'
        self.gpu_id = 0
        self.train_ratio = .6
        self.valid_ratio = .2
        self.test_ratio = .2
        self.batch_size = 256
        self.n_epochs = 10
        self.verbose = 2
        self.model_name = 'resnet'
        self.dataset_name = 'catdog'
        self.n_classes = 2
        # 1. train from scratch (random init)
        # 2. train from pretrained weights
        # 3. train with freezed pretrained weights
        self.freeze = True
        self.use_pretrained = True
        # <- 3

In [35]:
def set_parameter_requires_grad(model, freeze):
    for param in model.parameters():
        param.requires_grad = not freeze
        
def main(config):
    if config.verbose >= 2:
        print(config)
    device = torch.device('cpu') if config.gpu_id < 0 else torch.device('cuda:%d' % config.gpu_id)
    
    model = models.resnet34(pretrained=config.use_pretrained)
    set_parameter_requires_grad(model, config.freeze)

    n_features = model.fc.in_features
    model.fc = nn.Linear(n_features, config.n_classes)
    input_size = 224
    model = model.to(device)
    
    train_loader, valid_loader, test_loader = get_loaders(config, input_size)
    
    print("Train:", len(train_loader.dataset))
    print("Valid:", len(valid_loader.dataset))
    print("Test:", len(test_loader.dataset))
    
    optimizer = optim.Adam(model.parameters())
    crit = nn.CrossEntropyLoss()
    
    if config.verbose >= 2:
        print(model)
        print(optimizer)
        print(crit)
        
    trainer = Trainer_ignite(config)
    trainer.train(model, crit, optimizer, train_loader, valid_loader)

        
config= Config()
main(config)

Train: 15000
Valid: 5000
Test: 5000
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

  2%|#5                                                                                          | 1/59 [00:00…

Epoch 1 - |param|=8.80e+01 |g-param|=2.57e+00 loss=4.4048e-01 accuracy=0.7525


  5%|####6                                                                                       | 1/20 [00:00…

Validation - loss=1.7578e-01 accuracy=0.9260 best_loss=inf


...

Validation - loss=1.1771e-01 accuracy=0.9461 best_loss=9.8039e-02


  2%|#5                                                                                          | 1/59 [00:00…

Epoch 10 - |param|=8.80e+01 |g-param|=6.12e-01 loss=1.0979e-01 accuracy=0.9527


  5%|####6                                                                                       | 1/20 [00:00…

Validation - loss=1.0004e-01 accuracy=0.9596 best_loss=9.8039e-02
